In [3]:
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import sys
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from collections import Counter

# Import Data

In [114]:
train_values = pd.read_csv('Richters_Predictor_Modeling_Earthquake_Damage_-_Train_Values.csv', index_col='building_id')
train_labels = pd.read_csv('Richters_Predictor_Modeling_Earthquake_Damage_-_Train_Labels.csv', index_col='building_id')

test_values = pd.read_csv('Richters_Predictor_Modeling_Earthquake_Damage_-_Test_Values.csv', index_col='building_id')

In [115]:
#explore X variables
train_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,25,1335,1621,1,55,6,3,n,r,n,...,0,0,0,0,0,0,0,0,0,0
669485,17,715,2060,2,0,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
602512,17,51,8163,3,55,6,7,t,r,q,...,0,0,0,0,0,0,0,0,0,0


# Feature Engineering

In [116]:
train_values['geo_level_1_id']=pd.Categorical(train_values['geo_level_1_id'])
train_values['geo_level_1_id']

building_id
802906     6
28830      8
94947     21
590882    22
201944    11
          ..
688636    25
669485    17
602512    17
151409    26
747594    21
Name: geo_level_1_id, Length: 260601, dtype: category
Categories (31, int64): [0, 1, 2, 3, ..., 27, 28, 29, 30]

In [117]:
train_values['geo_level_2_id']=pd.Categorical(train_values['geo_level_2_id'])
train_values['geo_level_2_id']

building_id
802906     487
28830      900
94947      363
590882     418
201944     131
          ... 
688636    1335
669485     715
602512      51
151409      39
747594       9
Name: geo_level_2_id, Length: 260601, dtype: category
Categories (1414, int64): [0, 1, 3, 4, ..., 1424, 1425, 1426, 1427]

In [118]:
train_values['geo_level_3_id']=pd.Categorical(train_values['geo_level_3_id'])
train_values['geo_level_3_id']

building_id
802906    12198
28830      2812
94947      8973
590882    10694
201944     1488
          ...  
688636     1621
669485     2060
602512     8163
151409     1851
747594     9101
Name: geo_level_3_id, Length: 260601, dtype: category
Categories (11595, int64): [0, 1, 3, 5, ..., 12563, 12564, 12565, 12567]

In [119]:
train_labels['damage_grade']

building_id
802906    3
28830     2
94947     3
590882    2
201944    3
         ..
688636    2
669485    3
602512    3
151409    2
747594    3
Name: damage_grade, Length: 260601, dtype: int64

In [120]:

from category_encoders import TargetEncoder

encoder = TargetEncoder()
train_values[['geo_level_1_id','geo_level_2_id', 'geo_level_3_id']] = encoder.fit_transform(train_values[['geo_level_1_id','geo_level_2_id','geo_level_3_id']], train_labels['damage_grade'])


In [124]:
train_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2.161724,2.740741,2.837838,2,30,6,5,1,1,0,...,0,0,0,0,0,0,0,0,1,0
28830,2.485273,2.487437,2.062500,2,10,8,7,0,1,0,...,0,0,0,0,0,0,0,0,1,0
94947,2.563369,2.518750,2.580882,2,10,5,5,0,1,0,...,0,0,0,0,0,0,0,0,1,0
590882,2.000960,2.107317,2.096774,2,10,6,5,0,1,0,...,0,0,0,0,0,0,0,0,1,0
201944,2.337713,2.348748,2.368852,3,30,8,9,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,2.054054,1.931034,2.071429,1,55,6,3,0,1,0,...,0,0,0,1,0,0,0,0,1,0
669485,2.794480,2.931034,2.979592,2,0,6,5,0,1,0,...,0,0,0,0,0,0,0,0,1,0
602512,2.794480,2.894895,2.818182,3,55,6,7,0,1,0,...,0,0,0,0,0,0,0,0,1,0


# One Hot get dummy

In [122]:
train_values= pd.get_dummies(train_values)

In [125]:
train_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2.161724,2.740741,2.837838,2,30,6,5,1,1,0,...,0,0,0,0,0,0,0,0,1,0
28830,2.485273,2.487437,2.062500,2,10,8,7,0,1,0,...,0,0,0,0,0,0,0,0,1,0
94947,2.563369,2.518750,2.580882,2,10,5,5,0,1,0,...,0,0,0,0,0,0,0,0,1,0
590882,2.000960,2.107317,2.096774,2,10,6,5,0,1,0,...,0,0,0,0,0,0,0,0,1,0
201944,2.337713,2.348748,2.368852,3,30,8,9,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,2.054054,1.931034,2.071429,1,55,6,3,0,1,0,...,0,0,0,1,0,0,0,0,1,0
669485,2.794480,2.931034,2.979592,2,0,6,5,0,1,0,...,0,0,0,0,0,0,0,0,1,0
602512,2.794480,2.894895,2.818182,3,55,6,7,0,1,0,...,0,0,0,0,0,0,0,0,1,0


# Spliting Data

In [126]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_values, train_labels, test_size=0.25, random_state=42)

type(X_train)
type(y_train)

pandas.core.frame.DataFrame

In [ ]:
#LGBM

In [ ]:
params = {
      "cat_l2": 10,
      "cat_smooth": 10,
      "colsample_bytree": 0.8,
      "feature_fraction_bynode":  0.8,
      "learning_rate": 0.03,
      "max_depth": 6,
      "min_child_samples": 63,
      "min_data_per_group": 100,
      "n_estimators": 200,
      "num_leaves": 63,
      "path_smooth": 0,
      "reg_alpha":  0.05,
      "reg_lambda": 0.05,
      "subsample_freq": 1,
      "subsample": 0.8,
      "max_bin": 127,
      "extra_trees": False,
      "is_unbalance": False,
      "boosting_type": 'gbdt',
      "n_jobs": 1,
      "objective": "multi-class",
      "num_class":3,
      "verbosity": -1,
      "seed": 17,
}


# fit_params={'feature_name':"auto",
#            'categorical_feature':cat_cols}

In [ ]:
LGBM = LGBMClassifier(**params)

In [ ]:
LGBM.fit(X_train,y_train)

# Scaling

In [127]:
from sklearn.preprocessing import PowerTransformer
scaler = PowerTransformer(method='yeo-johnson')
scaler.fit(X_train)

X_train_te_yj=scaler.transform(X_train)
X_test_te_yj=scaler.transform(X_test)

In [130]:
X_train_te_yj

array([[-0.27095314, -0.54972601, -0.60353022, ..., -0.07499115,
         0.19607549, -0.10175587],
       [ 1.85244191,  1.60135727,  1.54402987, ..., -0.07499115,
         0.19607549, -0.10175587],
       [ 0.35259631, -0.20969954, -0.2026008 , ..., -0.07499115,
         0.19607549, -0.10175587],
       ...,
       [ 0.86706684, -0.26191942, -0.47564359, ..., -0.07499115,
         0.19607549, -0.10175587],
       [ 0.38792439, -0.18080863, -0.39373399, ..., -0.07499115,
         0.19607549, -0.10175587],
       [ 0.23694656, -0.60084242, -0.73915057, ..., -0.07499115,
         0.19607549, -0.10175587]])

# XGBoost

In [15]:
train_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2.161724,2.740741,2.837838,2,30,6,5,1,1,0,...,0,0,0,0,0,0,0,0,1,0
28830,2.485273,2.487437,2.062500,2,10,8,7,0,1,0,...,0,0,0,0,0,0,0,0,1,0
94947,2.563369,2.518750,2.580882,2,10,5,5,0,1,0,...,0,0,0,0,0,0,0,0,1,0
590882,2.000960,2.107317,2.096774,2,10,6,5,0,1,0,...,0,0,0,0,0,0,0,0,1,0
201944,2.337713,2.348748,2.368852,3,30,8,9,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,2.054054,1.931034,2.071429,1,55,6,3,0,1,0,...,0,0,0,1,0,0,0,0,1,0
669485,2.794480,2.931034,2.979592,2,0,6,5,0,1,0,...,0,0,0,0,0,0,0,0,1,0
602512,2.794480,2.894895,2.818182,3,55,6,7,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [532]:
# for col in ['land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type','other_floor_type','position','plan_configuration','legal_ownership_status']:
#     train_values[col] = train_values[col].astype('category')

In [38]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_values, train_labels, test_size=0.25, random_state=0)

type(X_train)
type(y_train)

pandas.core.frame.DataFrame

In [131]:
train_labels['damage_grade']

building_id
802906    3
28830     2
94947     3
590882    2
201944    3
         ..
688636    2
669485    3
602512    3
151409    2
747594    3
Name: damage_grade, Length: 260601, dtype: int64

In [132]:
y_train

,damage_grade
building_id,
184292,2
85957,3
227366,2
617536,2
406641,2
...,...
79888,3
442764,3
17256,2


In [133]:
y_train.value_counts()

damage_grade
2               111265
3                65408
1                18777
dtype: int64

In [134]:
y_train=y_train-1

In [135]:
y_train.value_counts()

damage_grade
1               111265
2                65408
0                18777
dtype: int64

In [136]:
import xgboost as xgb
xgb_clf = xgb.XGBClassifier(min_child_weight= 5, 
                            learning_rate= 0.1, 
                            gamma= 0.05, 
                            subsample= 0.8,
                            colsample_bytree= 0.3, 
                            colsample_bynode= 0.8,
                            colsample_bylevel= 0.8, 
                            max_depth = 20, 
                            n_estimators = 150, 
                            random_state=0)

xgb_clf.fit(X_train_te_yj, y_train)

y_pred_xgb = xgb_clf.predict(X_test_te_yj)

In [137]:
y_test.value_counts()

damage_grade
2               36994
3               21810
1                6347
dtype: int64

In [138]:
y_test=y_test-1

In [139]:
y_test.value_counts()

damage_grade
1               36994
2               21810
0                6347
dtype: int64

In [141]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
print(confusion_matrix(y_test, y_pred_xgb))

[[ 3514  2754    79]
 [ 1344 31315  4335]
 [  128  7435 14247]]


In [143]:

from sklearn.metrics import f1_score

print("F1 Score = {:.6f}".format(f1_score(y_test, y_pred_xgb, average='micro')))

F1 Score = 0.753265


In [1]:
feature_important = xgb_clf.get_booster().get_score(importance_type='weight')
keys = list(feature_important.keys())
values = list(feature_important.values())

data = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending=False)
data.nlargest(40, columns="score").plot(kind='barh', figsize = (20,10)) ## plot top 40 features

In [33]:
train_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2.161724,2.740741,2.837838,2,30,6,5,1,1,0,...,0,0,0,0,0,0,0,0,1,0
28830,2.485273,2.487437,2.062500,2,10,8,7,0,1,0,...,0,0,0,0,0,0,0,0,1,0
94947,2.563369,2.518750,2.580882,2,10,5,5,0,1,0,...,0,0,0,0,0,0,0,0,1,0
590882,2.000960,2.107317,2.096774,2,10,6,5,0,1,0,...,0,0,0,0,0,0,0,0,1,0
201944,2.337713,2.348748,2.368852,3,30,8,9,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,2.054054,1.931034,2.071429,1,55,6,3,0,1,0,...,0,0,0,1,0,0,0,0,1,0
669485,2.794480,2.931034,2.979592,2,0,6,5,0,1,0,...,0,0,0,0,0,0,0,0,1,0
602512,2.794480,2.894895,2.818182,3,55,6,7,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [32]:
train_labels

,damage_grade
building_id,
802906,2
28830,1
94947,2
590882,1
201944,2
...,...
688636,1
669485,2
602512,2


# Hyperparameter tuning

In [31]:
train_labels

,damage_grade
building_id,
802906,2
28830,1
94947,2
590882,1
201944,2
...,...
688636,1
669485,2
602512,2


In [22]:
train_labels=train_labels-1
train_labels

,damage_grade
building_id,
802906,2
28830,1
94947,2
590882,1
201944,2
...,...
688636,1
669485,2
602512,2


# Optuna on scaling data - tried a couple of Optuna Search but none of it works, some gives really good f1 on training dataset but the ranking is bad on testing data - maybe overfit. For Optuna, tried both Test-Split and Cross Validation

In [146]:
import numpy as np
import optuna

import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb


def objective(trial):
#     (data, target) = sklearn.datasets.load_breast_cancer(return_X_y=True)
#     train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
#     dtrain = xgb.DMatrix(train_x, label=train_y)
#     dvalid = xgb.DMatrix(valid_x, label=valid_y)
    
    #X_train, X_test, y_train, y_test = train_test_split(train_values, train_labels, test_size=0.25, random_state=0)
    dtrain = xgb.DMatrix(X_train_te_yj, label=y_train)
    dvalid = xgb.DMatrix(X_test_te_yj, label=y_test)
    
    
#     (min_child_weight= 5, learning_rate= 0.1, gamma= 0.05, subsample= 0.8,colsample_bytree= 0.3, colsample_bynode= 0.8,
#  colsample_bylevel= 0.8, max_depth = 20, n_estimators = 150, random_state=0)
    
    param = {
        "verbosity": 0,
        "objective": "multi:softprob", #"binary:logistic",
        'num_class': 3,
        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree","dart"]),
        
        #"eta": trial.suggest_float("eta", 0.1, 1.0, step=0.1),
        #"gamma": trial.suggest_float("gamma", 0.1, 1.0, step=0.1),
        #"max_depth": trial.suggest_int("max_depth", 1, 40, step=1),
        #"min_child_weight": trial.suggest_int("min_child_weight", 2, 10, step=1),
        
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0, step=0.1),
        
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0, step=0.1),
        "colsample_bynode": trial.suggest_float("colsample_bynode", 0.2, 1.0, step=0.1),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.2, 1.0, step=0.1),
        
        "n_estimators":150
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 40, step=1)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10, step=1)
        param["eta"] = trial.suggest_float("eta", 0.1, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 0.1, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    bst = xgb.train(param, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.argmax(preds, axis=1)
    f1 = sklearn.metrics.f1_score(y_test, pred_labels, average='micro')
    return f1


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2022-07-27 18:41:52,893] A new study created in memory with name: no-name-8b1fa3d6-d86e-4204-b21d-11868195fcac
[I 2022-07-27 18:42:15,487] Trial 0 finished with value: 0.7508250065233074 and parameters: {'booster': 'dart', 'subsample': 0.8, 'colsample_bytree': 0.7, 'colsample_bynode': 0.8, 'colsample_bylevel': 1.0, 'max_depth': 13, 'min_child_weight': 3, 'eta': 0.13523239476416887, 'gamma': 0.5800878126994489, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 0.0008358504058132508, 'skip_drop': 0.015025643865273955}. Best is trial 0 with value: 0.7508250065233074.
[I 2022-07-27 18:42:25,634] Trial 1 finished with value: 0.7402035271906802 and parameters: {'booster': 'dart', 'subsample': 0.7, 'colsample_bytree': 0.2, 'colsample_bynode': 1.0, 'colsample_bylevel': 0.9000000000000001, 'max_depth': 14, 'min_child_weight': 2, 'eta': 0.2598854730370526, 'gamma': 0.49290851516439177, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize

Number of finished trials:  38
Best trial:
  Value: 0.7530966523921351
  Params: 
    booster: gbtree
    subsample: 0.8
    colsample_bytree: 0.8
    colsample_bynode: 0.4
    colsample_bylevel: 0.7
    max_depth: 31
    min_child_weight: 7
    eta: 0.18690100164627022
    gamma: 0.17055074730041178
    grow_policy: depthwise


In [75]:
import numpy as np
import optuna

import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb


def objective(trial):
#     (data, target) = sklearn.datasets.load_breast_cancer(return_X_y=True)
#     train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
#     dtrain = xgb.DMatrix(train_x, label=train_y)
#     dvalid = xgb.DMatrix(valid_x, label=valid_y)
    
    X_train, X_test, y_train, y_test = train_test_split(train_values, train_labels, test_size=0.25, random_state=0)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_test, label=y_test)
    
    
#     (min_child_weight= 5, learning_rate= 0.1, gamma= 0.05, subsample= 0.8,colsample_bytree= 0.3, colsample_bynode= 0.8,
#  colsample_bylevel= 0.8, max_depth = 20, n_estimators = 150, random_state=0)
    
    param = {
        "verbosity": 0,
        "objective": "multi:softprob", #"binary:logistic",
        'num_class': 3,
        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        "colsample_bynode": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        "n_estimators":150
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 20, step=2)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    bst = xgb.train(param, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.argmax(preds, axis=1)
    f1 = sklearn.metrics.f1_score(y_test, pred_labels, average='micro')
    return f1


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2022-07-25 21:05:54,126] A new study created in memory with name: no-name-d70ba54a-81c6-496e-ab64-cf2274a2e7b6
[I 2022-07-25 21:06:12,955] Trial 0 finished with value: 0.7488449908673697 and parameters: {'booster': 'dart', 'lambda': 0.016336945826813973, 'alpha': 0.7556873818622448, 'subsample': 0.6433819045047714, 'colsample_bytree': 0.7364963310242272, 'max_depth': 13, 'min_child_weight': 7, 'eta': 1.6781710535514717e-05, 'gamma': 1.0799456852814074e-07, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 0.0018796200894601464, 'skip_drop': 2.0973390930930195e-06}. Best is trial 0 with value: 0.7488449908673697.
[I 2022-07-25 21:06:15,085] Trial 1 finished with value: 0.6170434835996378 and parameters: {'booster': 'gblinear', 'lambda': 0.006096105559017636, 'alpha': 3.772093235075156e-07, 'subsample': 0.5534953336493791, 'colsample_bytree': 0.20038564991342236}. Best is trial 0 with value: 0.7488449908673697.
[I 2022-07-25 21:06:23,923] Tri

Number of finished trials:  35
Best trial:
  Value: 0.7542938711608418
  Params: 
    booster: gbtree
    lambda: 2.224657768249692e-05
    alpha: 0.0025873341876641205
    subsample: 0.9929706426897845
    colsample_bytree: 0.8100061915286394
    max_depth: 17
    min_child_weight: 6
    eta: 1.6625714458378577e-07
    gamma: 1.3243875270440299e-05
    grow_policy: lossguide


In [ ]:
# booster: dart
#     lambda: 0.043123276598695484
#     alpha: 5.4792691523316414e-08
#     subsample: 0.862584217030565
#     colsample_bytree: 0.8105038547811492
#     max_depth: 18
#     min_child_weight: 2
#     eta: 0.00592408070917455
#     gamma: 0.003199247062638451
#     grow_policy: depthwise
#     sample_type: weighted
#     normalize_type: forest
#     rate_drop: 0.9688211129499148
#     skip_drop: 6.663111964129005e-05

In [49]:
import xgboost as xgb
xgb_clf = xgb.XGBClassifier(
    booster='dart',
    reg_lambda=0.043123276598695484,
    reg_alpha=5.4792691523316414e-08,
    subsample=0.862584217030565,
    colsample_bytree=0.8105038547811492,
    max_depth=18,
    min_child_weight=2,
    eta=0.00592408070917455,
    gamma=0.003199247062638451,
    grow_policy='depthwise',
    sample_type='weighted',
    normalize_type='forest',
    rate_drop=0.9688211129499148,
    skip_drop=6.663111964129005e-05,
    n_estimators = 150, 
    random_state=0)
xgb_clf.fit(X_train, y_train)

y_pred_xgb_tt = xgb_clf.predict(X_test)

In [54]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_xgb_tt))
from sklearn.metrics import f1_score

print("F1 Score = {:.6f}".format(f1_score(y_test, y_pred_xgb_tt, average='micro')))

              precision    recall  f1-score   support

           0       0.71      0.53      0.61      6246
           1       0.75      0.86      0.80     36968
           2       0.78      0.64      0.71     21937

    accuracy                           0.76     65151
   macro avg       0.75      0.68      0.70     65151
weighted avg       0.76      0.76      0.75     65151

F1 Score = 0.755752


In [ ]:
#   Value: 0.7542938711608418
#   Params: 
#     booster: gbtree
#     lambda: 2.224657768249692e-05
#     alpha: 0.0025873341876641205
#     subsample: 0.9929706426897845
#     colsample_bytree: 0.8100061915286394
#     max_depth: 17
#     min_child_weight: 6
#     eta: 1.6625714458378577e-07
#     gamma: 1.3243875270440299e-05
#     grow_policy: lossguide

In [97]:
import xgboost as xgb
xgb_clf = xgb.XGBClassifier(
    booster='gbtree',
    reg_lambda=2.224657768249692e-05,
    reg_alpha=0.0025873341876641205,
    subsample=0.9929706426897845,
    colsample_bytree=0.8100061915286394,
    max_depth=17,
    min_child_weight=6,
    eta=1.6625714458378577e-07,
    gamma= 1.3243875270440299e-05,
    grow_policy='lossguide',
    n_estimators = 150, 
    random_state=0)
xgb_clf.fit(X_train, y_train)

y_pred_xgb_tt = xgb_clf.predict(X_test)

In [98]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_xgb_tt))
from sklearn.metrics import f1_score

print("F1 Score = {:.6f}".format(f1_score(y_test, y_pred_xgb_tt, average='micro')))

              precision    recall  f1-score   support

           0       0.70      0.52      0.60      6246
           1       0.75      0.86      0.80     36968
           2       0.78      0.64      0.70     21937

    accuracy                           0.75     65151
   macro avg       0.74      0.67      0.70     65151
weighted avg       0.75      0.75      0.75     65151

F1 Score = 0.753342


# CV Integration

In [103]:
import os
import shutil

import optuna

import sklearn.metrics
import xgboost as xgb


SEED = 0
N_FOLDS = 5
CV_RESULT_DIR = "./xgboost_cv_results"

def f1_micro(predt, dtrain):
    y_true = dtrain.get_label()
    y_pred = np.argmax(predt, axis=1)
    return "f1-micro", f1_score(y_true, y_pred, average="micro")

def objective(trial):
    print("Trial {} started.".format(trial.number))
    #(data, target) = sklearn.datasets.load_breast_cancer(return_X_y=True)
    dtrain = xgb.DMatrix(train_values, label=train_labels)

    param = {
#         "verbosity": 0,
#         "objective": "multi:softprob",
#         'num_class': 3,
#         "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
#         "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
#         "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
#         # sampling ratio for training data.
#         "subsample": trial.suggest_float("subsample", 0.2, 1.0),
#         # sampling according to each tree.
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        
        
        "verbosity": 0,
        "objective": "multi:softprob", #"binary:logistic",
        'num_class': 3,
        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        "colsample_bynode": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        "n_estimators":150
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    xgb_cv_results = xgb.cv(
        params=param,
        dtrain=dtrain,
        num_boost_round=10000,
        nfold=N_FOLDS,
        stratified=True,
        early_stopping_rounds=100,
        seed=SEED,
        verbose_eval=True,
        feval=f1_micro
    )

    # Set n_estimators as a trial attribute; Accessible via study.trials_dataframe().
    trial.set_user_attr("n_estimators", len(xgb_cv_results))

    # Save cross-validation results.
    filepath = os.path.join(CV_RESULT_DIR, "{}.csv".format(trial.number))
    xgb_cv_results.to_csv(filepath, index=False)
    # Extract the best score.
    best_score = xgb_cv_results["test-f1-micro-mean"].values[-1]
    print("Trial {} ended with best score {}.".format(trial.number, best_score))
    return best_score



if not os.path.exists(CV_RESULT_DIR):
    os.mkdir(CV_RESULT_DIR)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20, timeout=600)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

print("  Number of estimators: {}".format(trial.user_attrs["n_estimators"]))

shutil.rmtree(CV_RESULT_DIR)

[I 2022-07-26 09:13:34,594] A new study created in memory with name: no-name-f4df5fc3-dcef-41de-902a-04ed0530c600


Trial 0 started.
[0]	train-mlogloss:1.09861+0.00000	train-f1-micro:0.61567+0.02338	test-mlogloss:1.09861+0.00000	test-f1-micro:0.61573+0.02342
[1]	train-mlogloss:1.09861+0.00000	train-f1-micro:0.58912+0.04061	test-mlogloss:1.09861+0.00000	test-f1-micro:0.58882+0.03997
[2]	train-mlogloss:1.09861+0.00000	train-f1-micro:0.59265+0.02907	test-mlogloss:1.09861+0.00000	test-f1-micro:0.59286+0.02934
[3]	train-mlogloss:1.09861+0.00000	train-f1-micro:0.56891+0.00001	test-mlogloss:1.09861+0.00000	test-f1-micro:0.56891+0.00001
[4]	train-mlogloss:1.09861+0.00000	train-f1-micro:0.56951+0.00117	test-mlogloss:1.09861+0.00000	test-f1-micro:0.56951+0.00113
[5]	train-mlogloss:1.09861+0.00000	train-f1-micro:0.56891+0.00000	test-mlogloss:1.09861+0.00000	test-f1-micro:0.56891+0.00001
[6]	train-mlogloss:1.09861+0.00000	train-f1-micro:0.56891+0.00000	test-mlogloss:1.09861+0.00000	test-f1-micro:0.56891+0.00001
[7]	train-mlogloss:1.09861+0.00000	train-f1-micro:0.56891+0.00000	test-mlogloss:1.09861+0.00000	test-

[I 2022-07-26 09:15:34,136] Trial 0 finished with value: 0.5689120000000001 and parameters: {'booster': 'gbtree', 'lambda': 1.412514835852336e-07, 'alpha': 0.08050048110363223, 'subsample': 0.9627953098762378, 'colsample_bytree': 0.2629083978115778, 'max_depth': 2, 'min_child_weight': 10, 'eta': 2.225381018067502e-06, 'gamma': 6.267509320628424e-06, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.5689120000000001.


Trial 0 ended with best score 0.5689120000000001.
Trial 1 started.
[0]	train-mlogloss:0.94985+0.00007	train-f1-micro:0.56891+0.00000	test-mlogloss:0.94985+0.00010	test-f1-micro:0.56891+0.00001
[1]	train-mlogloss:0.92384+0.00009	train-f1-micro:0.56891+0.00000	test-mlogloss:0.92385+0.00019	test-f1-micro:0.56891+0.00001
[2]	train-mlogloss:0.91584+0.00013	train-f1-micro:0.56891+0.00000	test-mlogloss:0.91585+0.00027	test-f1-micro:0.56891+0.00001
[3]	train-mlogloss:0.91314+0.00017	train-f1-micro:0.56891+0.00000	test-mlogloss:0.91316+0.00038	test-f1-micro:0.56891+0.00001
[4]	train-mlogloss:0.91229+0.00022	train-f1-micro:0.56891+0.00000	test-mlogloss:0.91231+0.00052	test-f1-micro:0.56891+0.00001
[5]	train-mlogloss:0.91170+0.00025	train-f1-micro:0.56891+0.00000	test-mlogloss:0.91173+0.00065	test-f1-micro:0.56891+0.00001
[6]	train-mlogloss:0.91129+0.00027	train-f1-micro:0.56891+0.00000	test-mlogloss:0.91133+0.00075	test-f1-micro:0.56891+0.00001
[7]	train-mlogloss:0.91083+0.00031	train-f1-micro:0

[I 2022-07-26 09:17:43,721] Trial 1 finished with value: 0.5689120000000001 and parameters: {'booster': 'gblinear', 'lambda': 1.2354799867557716e-08, 'alpha': 0.12066925592480995, 'subsample': 0.6903104844990988, 'colsample_bytree': 0.4852429738576032}. Best is trial 0 with value: 0.5689120000000001.


Trial 1 ended with best score 0.5689120000000001.
Trial 2 started.
[0]	train-mlogloss:1.09861+0.00000	train-f1-micro:0.70663+0.00818	test-mlogloss:1.09861+0.00000	test-f1-micro:0.70651+0.00894
[1]	train-mlogloss:1.09860+0.00000	train-f1-micro:0.71113+0.00903	test-mlogloss:1.09860+0.00000	test-f1-micro:0.71111+0.00963
[2]	train-mlogloss:1.09860+0.00000	train-f1-micro:0.71225+0.00848	test-mlogloss:1.09860+0.00000	test-f1-micro:0.71170+0.00926
[3]	train-mlogloss:1.09859+0.00000	train-f1-micro:0.71174+0.00940	test-mlogloss:1.09859+0.00000	test-f1-micro:0.71156+0.01025
[4]	train-mlogloss:1.09859+0.00000	train-f1-micro:0.71347+0.00554	test-mlogloss:1.09859+0.00000	test-f1-micro:0.71328+0.00603
[5]	train-mlogloss:1.09858+0.00000	train-f1-micro:0.71255+0.00433	test-mlogloss:1.09858+0.00000	test-f1-micro:0.71231+0.00499
[6]	train-mlogloss:1.09858+0.00000	train-f1-micro:0.71331+0.00287	test-mlogloss:1.09858+0.00000	test-f1-micro:0.71346+0.00350
[7]	train-mlogloss:1.09858+0.00000	train-f1-micro:0

[I 2022-07-26 12:07:16,692] Trial 2 finished with value: 0.7065092 and parameters: {'booster': 'dart', 'lambda': 0.3482952672119896, 'alpha': 0.008235400789063875, 'subsample': 0.4383682706914903, 'colsample_bytree': 0.5395507059518736, 'max_depth': 3, 'min_child_weight': 5, 'eta': 9.552835409341935e-06, 'gamma': 6.0734393803342895e-06, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 0.002168497857778161, 'skip_drop': 0.2193711947698926}. Best is trial 2 with value: 0.7065092.


Trial 2 ended with best score 0.7065092.
Number of finished trials:  3
Best trial:
  Value: 0.7065092
  Params: 
    booster: dart
    lambda: 0.3482952672119896
    alpha: 0.008235400789063875
    subsample: 0.4383682706914903
    colsample_bytree: 0.5395507059518736
    max_depth: 3
    min_child_weight: 5
    eta: 9.552835409341935e-06
    gamma: 6.0734393803342895e-06
    grow_policy: lossguide
    sample_type: weighted
    normalize_type: tree
    rate_drop: 0.002168497857778161
    skip_drop: 0.2193711947698926
  Number of estimators: 1


In [104]:
import os
import shutil

import optuna

import sklearn.metrics
import xgboost as xgb


SEED = 0
N_FOLDS = 5
CV_RESULT_DIR = "./xgboost_cv_results"

def f1_micro(predt, dtrain):
    y_true = dtrain.get_label()
    y_pred = np.argmax(predt, axis=1)
    return "f1-micro", f1_score(y_true, y_pred, average="micro")

def objective(trial):
    print("Trial {} started.".format(trial.number))
    #(data, target) = sklearn.datasets.load_breast_cancer(return_X_y=True)
    dtrain = xgb.DMatrix(train_values, label=train_labels)

    param = {
#         "verbosity": 0,
#         "objective": "multi:softprob",
#         'num_class': 3,
#         "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
#         "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
#         "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
#         # sampling ratio for training data.
#         "subsample": trial.suggest_float("subsample", 0.2, 1.0),
#         # sampling according to each tree.
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        
#     (min_child_weight= 5, learning_rate= 0.1, gamma= 0.05, subsample= 0.8,colsample_bytree= 0.3, colsample_bynode= 0.8,
#  colsample_bylevel= 0.8, max_depth = 20, n_estimators = 150, random_state=0)
        
        "verbosity": 0,
        "objective": "multi:softprob", #"binary:logistic",
        'num_class': 3,
        # use exact for small dataset.
        #"tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree"]),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        "colsample_bynode": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        "n_estimators":150
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 20)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        #param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

#     if param["booster"] == "dart":
#         param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
#         param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
#         param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
#         param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    xgb_cv_results = xgb.cv(
        params=param,
        dtrain=dtrain,
        num_boost_round=10000,
        nfold=N_FOLDS,
        stratified=True,
        early_stopping_rounds=100,
        seed=SEED,
        verbose_eval=True,
        feval=f1_micro
    )

    # Set n_estimators as a trial attribute; Accessible via study.trials_dataframe().
    trial.set_user_attr("n_estimators", len(xgb_cv_results))

    # Save cross-validation results.
    filepath = os.path.join(CV_RESULT_DIR, "{}.csv".format(trial.number))
    xgb_cv_results.to_csv(filepath, index=False)
    # Extract the best score.
    best_score = xgb_cv_results["test-f1-micro-mean"].values[-1]
    print("Trial {} ended with best score {}.".format(trial.number, best_score))
    return best_score



if not os.path.exists(CV_RESULT_DIR):
    os.mkdir(CV_RESULT_DIR)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20, timeout=600)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

print("  Number of estimators: {}".format(trial.user_attrs["n_estimators"]))

shutil.rmtree(CV_RESULT_DIR)

[I 2022-07-26 12:10:32,364] A new study created in memory with name: no-name-22430ed6-08e1-4d14-8853-d3d71c0b3f24


Trial 0 started.
[0]	train-mlogloss:1.09861+0.00000	train-f1-micro:0.74886+0.00055	test-mlogloss:1.09861+0.00000	test-f1-micro:0.73812+0.00162
[1]	train-mlogloss:1.09861+0.00000	train-f1-micro:0.75557+0.00059	test-mlogloss:1.09861+0.00000	test-f1-micro:0.74433+0.00169
[2]	train-mlogloss:1.09861+0.00000	train-f1-micro:0.75878+0.00105	test-mlogloss:1.09861+0.00000	test-f1-micro:0.74700+0.00148
[3]	train-mlogloss:1.09861+0.00000	train-f1-micro:0.76027+0.00070	test-mlogloss:1.09861+0.00000	test-f1-micro:0.74777+0.00130
[4]	train-mlogloss:1.09861+0.00000	train-f1-micro:0.76119+0.00057	test-mlogloss:1.09861+0.00000	test-f1-micro:0.74870+0.00126
[5]	train-mlogloss:1.09861+0.00000	train-f1-micro:0.76146+0.00040	test-mlogloss:1.09861+0.00000	test-f1-micro:0.74913+0.00148
[6]	train-mlogloss:1.09861+0.00000	train-f1-micro:0.76186+0.00029	test-mlogloss:1.09861+0.00000	test-f1-micro:0.74968+0.00174
[7]	train-mlogloss:1.09861+0.00000	train-f1-micro:0.76208+0.00034	test-mlogloss:1.09861+0.00000	test-

[I 2022-07-26 15:16:11,757] Trial 0 finished with value: 0.7381208 and parameters: {'booster': 'dart', 'subsample': 0.2299022088951193, 'colsample_bytree': 0.9463986641275566, 'max_depth': 16, 'min_child_weight': 8, 'eta': 1.0237178130662043e-06, 'gamma': 1.5171827817332288e-07, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 0.10421127272740319, 'skip_drop': 0.001686586405607477}. Best is trial 0 with value: 0.7381208.


Trial 0 ended with best score 0.7381208.
Number of finished trials:  1
Best trial:
  Value: 0.7381208
  Params: 
    booster: dart
    subsample: 0.2299022088951193
    colsample_bytree: 0.9463986641275566
    max_depth: 16
    min_child_weight: 8
    eta: 1.0237178130662043e-06
    gamma: 1.5171827817332288e-07
    grow_policy: lossguide
    sample_type: weighted
    normalize_type: tree
    rate_drop: 0.10421127272740319
    skip_drop: 0.001686586405607477
  Number of estimators: 1


# Make prediction and save submission

In [79]:
test_values = pd.read_csv('Richters_Predictor_Modeling_Earthquake_Damage_-_Test_Values.csv', index_col='building_id')

In [80]:
test_values = pd.get_dummies(test_values)

In [81]:
test_values['geo_level_1_id']=pd.Categorical(test_values['geo_level_1_id'])
test_values['geo_level_1_id']

building_id
300051     17
99355       6
890251     22
745817     26
421793     17
           ..
310028      4
663567     10
1049160    22
442785      6
501372     26
Name: geo_level_1_id, Length: 86868, dtype: category
Categories (31, int64): [0, 1, 2, 3, ..., 27, 28, 29, 30]

In [82]:
test_values['geo_level_2_id']=pd.Categorical(test_values['geo_level_2_id'])
test_values['geo_level_2_id']

building_id
300051      596
99355       141
890251       19
745817       39
421793      289
           ... 
310028      605
663567     1407
1049160    1136
442785     1041
501372       36
Name: geo_level_2_id, Length: 86868, dtype: category
Categories (1364, int64): [0, 1, 3, 4, ..., 1424, 1425, 1426, 1427]

In [83]:
test_values['geo_level_3_id']=pd.Categorical(test_values['geo_level_3_id'])
test_values['geo_level_3_id']

building_id
300051     11307
99355      11987
890251     10044
745817       633
421793      7970
           ...  
310028      3623
663567     11907
1049160     7712
442785       912
501372      6436
Name: geo_level_3_id, Length: 86868, dtype: category
Categories (10213, int64): [0, 2, 3, 4, ..., 12564, 12565, 12566, 12567]

In [84]:
# train_values['geo_level_1_id']=pd.Categorical(train_values['geo_level_1_id'])
# train_values['geo_level_1_id']

In [85]:
# train_values['geo_level_2_id']=pd.Categorical(train_values['geo_level_2_id'])
# train_values['geo_level_2_id']

In [86]:
# train_values['geo_level_3_id']=pd.Categorical(train_values['geo_level_3_id'])
# train_values['geo_level_3_id']

In [87]:
test_values[['geo_level_1_id','geo_level_2_id', 'geo_level_3_id']] = encoder.transform(test_values[['geo_level_1_id','geo_level_2_id','geo_level_3_id']])


In [88]:
test_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
building_id,,,,,,,,,,,,,,,,,,,,,
300051,2.794480,2.705036,2.631579,3,20,7,6,0,1,0,...,0,0,0,0,0,0,0,0,1,0
99355,2.161724,2.180851,2.000080,2,25,13,5,0,1,0,...,0,0,0,0,0,0,0,0,1,0
890251,2.000960,2.175000,2.795140,2,5,4,5,0,1,0,...,0,0,0,0,0,0,0,0,1,0
745817,1.730887,1.532194,1.437788,1,0,19,3,0,0,0,...,0,0,0,0,0,0,0,0,1,0
421793,2.794480,2.689119,2.823529,3,15,8,7,0,1,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310028,2.162136,2.122642,1.861111,3,70,20,6,0,1,0,...,0,0,0,0,0,0,0,0,0,1
663567,2.341954,2.337176,2.722222,3,25,6,7,1,1,1,...,0,0,0,0,0,0,0,0,1,0
1049160,2.000960,2.019704,1.909106,1,50,3,3,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [89]:
predictions = xgb_clf.predict(test_values)

In [90]:
predictions=predictions+1

In [91]:
predictions

array([3, 2, 3, ..., 2, 2, 1])

In [92]:
submission_format = pd.read_csv('Richters_Predictor_Modeling_Earthquake_Damage_-_Submission_Format.csv', index_col='building_id')

In [93]:
my_submission = pd.DataFrame(data=predictions,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [94]:
my_submission

,damage_grade
building_id,
300051,3
99355,2
890251,3
745817,1
421793,3
...,...
310028,2
663567,3
1049160,2


In [95]:
my_submission.to_csv('submission_3.csv')

In [96]:
!head submission_1.csv

building_id,damage_grade
300051,3
99355,2
890251,2
745817,1
421793,3
871976,2
691228,1
896100,3
343471,2
